In [21]:
##### importing custom modules from the projects folder
import sys
from pathlib import Path
# Add project root to sys.path
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(PROJECT_ROOT))
import modules.scrapers as scrapers
import modules.helperModule as hf
##### --------------
from datetime import date

# ====================================
#                    PARAMS 
# ====================================
league = 'nfl'
export = True # export to csv files
today = date.today()
season = 2025
week = 0
inseason = False  # False for running proj, and rank scrapes in the offseason
#week = int(input("What week is it >>>>>> ? "))
print('Run date:',today, "\nWeek:", week, '\ninseason run:', inseason, '\nexport files:', export)
# ====================================

scraper = scrapers.scrapers(
    season = season,
    week = week,
    today = today
)

Run date: 2025-08-26 
Week: 0 
inseason run: False 
export files: True


# SCRAPE DATA
# =============

### INSEASON - Actual game scores and Game betting lines
##### run after the last game of the week before TNF game

In [ ]:
scraper.cbs_game_scores(inseason = inseason, export = export)
scraper.bp_lines(export = export)

### INSEASON or OFFSEASON - Projections and Rankings
##### offseason - run at any time in the offseason once sites put them up
##### inseason - run anytime Tues-Thurs before TNF to get the proj or ranks before cames start

In [22]:
# in- and off-season
scraper.cbs_projections(inseason = inseason, export = export)
scraper.espn_projections(inseason = inseason, export = export)
scraper.nfl_projections(inseason = inseason, export = export)

(1111, 60)

In [23]:
# in- and off-season
scraper.ffp_ecr_rankings(inseason = inseason, export = export)
scraper.espn_rankings(inseason = inseason, export = export)
scraper.nfl_rankings(inseason = inseason, export = export)

c:\Users\jrbrz\Desktop\projects\projects\fantasyfootball\modules\scrapers.py:1547: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_espn_ranking = pd.concat([df_espn_ranking, temp_df], axis = 0, ignore_index=True)


(384, 13)

###  OFFSEASON - ADPs

In [24]:
scraper.cbs_adp(export = export)
scraper.ffp_adp(export = export)
scraper.espn_adp(export = export)

(500, 10)

### OFFSEASON - player full season yard and td props 

In [25]:
scraper.bp_season_player_total_props()

total_rushing_yds 1 <Response [200]>
made soup
total_rushing_yds 2 <Response [200]>
made soup
total_rushing_yds 3 <Response [200]>
made soup
total_rushing_yds 4 <Response [200]>
made soup
total_rushing_yds 5 <Response [200]>
made soup
total_rushing_yds 6 <Response [200]>
made soup
total_rushing_yds 7 <Response [200]>
made soup
total_rushing_yds 8 <Response [200]>
made soup
total_rushing_yds 9 <Response [200]>
made soup
total_rushing_yds 10 <Response [200]>
made soup
total_rushing_yds 11 <Response [200]>
made soup
total_rushing_yds 12 <Response [200]>
made soup
total_rushing_tds 1 <Response [200]>
made soup
total_rushing_tds 2 <Response [200]>
made soup
total_rushing_tds 3 <Response [200]>
made soup
total_rushing_tds 4 <Response [200]>
made soup
total_rushing_tds 5 <Response [200]>
made soup
total_rushing_tds 6 <Response [200]>
made soup
total_rushing_tds 7 <Response [200]>
made soup
total_rushing_tds 8 <Response [200]>
made soup
total_rushing_tds 9 <Response [200]>
made soup
total_rush

# PROCESS SCRAPED DATA

In [30]:
# hit db to standardize player, team, outlet, etc... ids 
scraper.generate_id_maps()

query successful
query successful
query successful
query successful
query successful
query successful


In [31]:
scraper.process_projections()

c:\Users\jrbrz\Desktop\projects\projects\fantasyfootball\modules\scrapers.py:2776: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_load_proj = pd.concat([df_load_proj, temp])
c:\Users\jrbrz\Desktop\projects\projects\fantasyfootball\modules\scrapers.py:2763: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_missing_players_proj = pd.concat(


0 missing players..
query successful
query successful
updates completed...


In [32]:
scraper.process_rankings()

no missing players


c:\Users\jrbrz\Desktop\projects\projects\fantasyfootball\modules\scrapers.py:2641: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_load_rank = pd.concat([df_load_rank, temp])


In [33]:
scraper.process_adps()

cbs_adp_2025-00_2025-08-26.csv
espn_adp_2025-00_2025-08-26.csv
fp_adp_2025-00_2025-08-26.csv
raw
no missing players


c:\Users\jrbrz\Desktop\projects\projects\fantasyfootball\modules\scrapers.py:2908: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_load_adp = pd.concat([df_load_adp, temp])


In [ ]:
scraper.process_game_lines(bettingTableName = 'betting')

In [34]:
# only need to run when season total props are scraped.
scraper.process_season_player_total_props()

# load

In [35]:
print(
    'PROJ:', scraper.processed_dfs['projections'].shape,
    '\nRANK:', scraper.processed_dfs['rankings'].shape,
    '\nADP:', scraper.processed_dfs['adps'].shape)

PROJ: (1544, 56) 
RANK: (2953, 10) 
ADP: (2055, 6)


In [40]:
# =============================================
# remove nan player ids if the loads fail below
# COME BACK TO HERE IF THE 3 expor_Database() calls below fail for nans 
# =============================================
table = 'adps'

scraper.processed_dfs[table] = scraper.processed_dfs[table][
    scraper.processed_dfs[table]['playerId'].notna()
]
scraper.processed_dfs[table].shape

(2055, 6)

In [37]:
hf.export_database(
    dataframe=scraper.processed_dfs['projections'],
    database_table='projection', 
    connection_string=None, 
    if_exists='append'
)

successfully added data to projection


In [ ]:
hf.export_database(
    dataframe=scraper.processed_dfs['rankings'],
    database_table='ranking', 
    connection_string=None, 
    if_exists='append'
)

In [ ]:
hf.export_database(
    dataframe=scraper.processed_dfs['adps'],
    database_table='adp', 
    connection_string=None, 
    if_exists='append'
)

In [ ]:

hf.export_database(
    dataframe=scraper.processed_dfs['lines'], 
    database_table='betting', 
    connection_string=None, 
    if_exists='append'
)

In [ ]:
hf.export_database(
    dataframe=scraper.processed_dfs['seasonprops'], 
    database_table='odds_season_totals', 
    connection_string=None, 
    if_exists='append'
)

# scratch

In [ ]:
load sleeper